## Power BI

## ElasticSearch

+ APIまとめ1:https://medium.com/hello-elasticsearch/elasticsearch-api-83760ce1424b
+ APIまとめ2:https://blog.mosuke.tech/entry/2018/01/14/elasticsearch-api/


### API

curl -H "Content-Type: application/json" -XPUT localhost:9200/electric_power --data-binary @mapping.json
<br>

curl -XPOST http://localhost:9200/_bulk -H "Content-type: application/json"   --data-binary @dataset_1.json
<br>

curl -XDELETE "localhost:9200/*"


## Kibana

+ 日本語化：https://qiita.com/kouji-kojima/items/7413d5e6ec4b7e82373b


## MongoDB

### MongoDBとは
+ humongous（とてつもなく大きい）が由来
+ javascriptで作成されている
+ ドキュメント指向データベース（スキーマレス）
+ テーブル結合はできる
+ 水平スケールアウトが容易（シャーディング）
+ トランザクションがない（v4.0から一応ある様子）
+ wiredTigerストレージエンジンを採用（ロック：ドキュメント単位(書込みが早い)、キャッシュサイズ：指定可能、データ圧縮：可能）

### 設計指針

以下のアンチパターンは避ける
+ 同じフィールド名でデータ型が異なる
+ 1つのコレクションに2つの役割を持つ
+ 深いネストがある
+ 多対多の設計

デザインパターン
+ シャーディング前提で設計する。（どれをシャードキーにするかを念頭に入れる）
+ シャードキーのアンチパターン
    + 検索に使用しないフィールド
    + 単調増加、単調減少するフィールド（例：日付をキーにすると、常に書込みが特定のサーバに集中する。）
    + カーディナリティが低すぎるフィールド（例：性別等を指定してしまうとチャンクサイズが大きくなり分割できなくなる。）
+ シャードキーのデザインパターン
    + 複合キーを指定する、一つのキーを指定するといずれはチャンクが最大に達する。
    + 例：投稿ID,ユーザID,投稿日時,投稿内容,等の場合にはユーザIDを指定する。投稿ID、投稿日時は単調増加する
    + キーの指定順番は注意が必要。例：ユーザID,投稿年月をシャードキーする場合に（投稿年月、ユーザID）とする

### 設定

+ コンフィグ, もしくは引数でデータ、ログの保存先を指定可能

In [ ]:
mongod --dbpath c:\mongodb\data --logpath c:\mongodb\logs\mongodb.log

### コマンド

+ use test: testデータベース作成、スイッチ
+ show dbs: 
+ show collections:
+ db.createCollection("employee"):コレクション作成
+ db.employee.insertOne({name:"Alice",age:30}):ドキュメント作成、ここで型指定も可能、NULL可、Validation機能あり
+ db.employee.insertMany({name:"Alice",age:30},{name:"Bob",age:20}):ドキュメント作成
+ db.employee.find():検索
+ db.employee.find({}, {_id:0}):全件検索、_id列を非表示
+ db.employee.find({name:"Alice"},{_id:0, name:1} 
+ db.stats():統計情報の表示
+ db.dropDatabase():DB削除

### インデックス
+ B-Treeアルゴリズム
+ 結合キー、シャードキーにはインデックスを付与する
+ 無駄なインデックスは登録・更新・削除を遅くする。プロファイラを設定して監視。

In [ ]:
# クエリパフォーマンスの監視
use test
db.setPrifilingLevel(1,1) #スロークエリのみ出力, 敷値は1msec

### その他設定
+ configファイルに記載。
+ destination:
+ path:
+ logAppend:
+ logRotate:rename(default)
+ dbPath:
+ journal enabled:true(default) 50msec毎に書込発生。
+ cacheSizeGB:memory-size*0.5-1(default)
+ blockCompressor:none, snappy(default), zlib
+ bindIp:MongoDBに接続可能なIP
+ port:
+ maxIncomingConnections:最大同時接続数 65536(default)

### レプリカ
+ 導入初期はレプリカセットのみの使用を検討してみる
+ primary, secondary, arbiterの3台が最小構成。arbiterはprimaryの投票だけを行う。3台はpingで相互に死活監視を行う。
+ データ追加時にwriteConcernオプションを付与する事で書込保障機能を利用可能
+ 読取負荷分散はdb.collection.find().readPref({mode:"primary","secondary","nearest"})等として利用できる
+ 同期の仕組み
    1. primaryへの書き込み発生
    1. primaryに書き込まれたデータや登録日時をoplogに書き込む
    1. secondaryがprimaryのoplogを非同期に確認。差分をsecondaryのoplogに書き込む。
    1. secondaryがoplogからデータを取得して同期する
+ oplogのサイジング
    + oplogは上限がデフォルトで指定されている。
    + <span style="color: red; ">超過分は古いデータから消去される。(障害時の同期処理が正常に完了しない等の問題が発生する)</span>
    + 1時間分の本番データを入れて、8時間分のoplogサイジングする事が推奨されている

### シャーディング
+ キーを指定してチャンクを作成する
+ <span style="color: red; ">チャンクサイズは64MBが最大。超過分は同じチャンクに書き込まれ続ける。パフォーマンス悪化の原因。</span>
+ オートバランシング中にデータ重複が発生し、クエリ結果に不整合が生じる。対応策は以下・・・？
    + ゾーン指定：運用が大変
    + オートバランシングの停止：sh.isBalancerRunning(), sh.stopBalancer(timeout, interval)

### セキュリティ
+ ユーザ作成する程度


In [ ]:
mongo localhost:20000/admin -u "user" -p "password"

### バックアップ

+ ファイルコピー。データが入ったファイルを丸ごとコピーする
+ mongodumpを使用する。BSONに変換されてバックアップ、リストアに時間が掛かる。<span style="color: red; ">シャーディングを構築している時などに使用する。</span>